In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sqlalchemy import create_engine
import warnings

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

import statsmodels.api as sm

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [3]:
data_loc = "data/Iowa LAUS Data 2020-07.csv"

<IPython.core.display.Javascript object>

In [4]:
iowa = pd.read_csv(data_loc)

<IPython.core.display.Javascript object>

In [5]:
iowa.head()

,STFIPS,AREATYNAME,AREANAME,YEAR,MONTH,ADJUSTED,PRELIM,LABORFORCE,EMP,UNEMP,UNEMPRATE,LABFORCE_PARTRATE
0,19,Cities,Mason City,2004,July,0,0,16200,15500,800,4.6,NaN
1,19,Cities,Mason City,2004,August,0,0,16000,15200,800,4.8,NaN
2,19,Cities,Mason City,2004,September,0,0,15700,15000,700,4.6,NaN
3,19,Cities,Mason City,2004,October,0,0,15900,15200,700,4.4,NaN
4,19,Cities,Mason City,2004,November,0,0,15900,15100,800,4.8,NaN


<IPython.core.display.Javascript object>

In [8]:
iowa_story = iowa[iowa["AREANAME"] == "Story County"]

<IPython.core.display.Javascript object>

In [12]:
iowa_story = iowa_story[iowa_story["YEAR"] <= 2010]
iowa_story = iowa_story[iowa_story["YEAR"] >= 2006]

<IPython.core.display.Javascript object>

In [13]:
iowa_story.head()

,STFIPS,AREATYNAME,AREANAME,YEAR,MONTH,ADJUSTED,PRELIM,LABORFORCE,EMP,UNEMP,UNEMPRATE,LABFORCE_PARTRATE
35554,19,County,Story County,2006,January,0,0,46700,45400,1400,2.9,NaN
35555,19,County,Story County,2006,February,0,0,48000,46600,1400,2.9,NaN
35556,19,County,Story County,2006,March,0,0,47500,46200,1300,2.7,NaN
35557,19,County,Story County,2006,April,0,0,48200,47000,1100,2.4,NaN
35558,19,County,Story County,2006,May,0,0,47600,46500,1200,2.5,NaN


<IPython.core.display.Javascript object>

In [14]:
unemp_rates = iowa_story[["MONTH", "YEAR", "UNEMPRATE"]]

<IPython.core.display.Javascript object>

In [23]:
dir(calendar)

['Calendar',
 'EPOCH',
 'FRIDAY',
 'February',
 'HTMLCalendar',
 'IllegalMonthError',
 'IllegalWeekdayError',
 'January',
 'LocaleHTMLCalendar',
 'LocaleTextCalendar',
 'MONDAY',
 'SATURDAY',
 'SUNDAY',
 'THURSDAY',
 'TUESDAY',
 'TextCalendar',
 'WEDNESDAY',
 '_EPOCH_ORD',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_colwidth',
 '_locale',
 '_localized_day',
 '_localized_month',
 '_spacing',
 'c',
 'calendar',
 'datetime',
 'day_abbr',
 'day_name',
 'different_locale',
 'error',
 'firstweekday',
 'format',
 'formatstring',
 'isleap',
 'leapdays',
 'main',
 'mdays',
 'month',
 'month_abbr',
 'month_name',
 'monthcalendar',
 'monthlen',
 'monthrange',
 'nextmonth',
 'prcal',
 'prevmonth',
 'prmonth',
 'prweek',
 'repeat',
 'setfirstweekday',
 'sys',
 'timegm',
 'week',
 'weekday',
 'weekheader']

<IPython.core.display.Javascript object>

In [24]:
import calendar

month_key = {v: k for k, v in enumerate(calendar.month_name)}

<IPython.core.display.Javascript object>

In [25]:
month_key

{'': 0,
 'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12}

<IPython.core.display.Javascript object>

In [27]:
unemp_rates["MONTH"] = unemp_rates["MONTH"].replace(to_replace=month_key)

<IPython.core.display.Javascript object>

In [29]:
unemp_rates.to_csv("data/unemp.csv", index=True)

<IPython.core.display.Javascript object>

In [28]:
unemp_rates

,MONTH,YEAR,UNEMPRATE
35554,1,2006,2.9
35555,2,2006,2.9
35556,3,2006,2.7
35557,4,2006,2.4
35558,5,2006,2.5
35559,6,2006,3.1
35560,7,2006,2.7
35561,8,2006,2.6
35562,9,2006,2.5
35563,10,2006,2.4


<IPython.core.display.Javascript object>